In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm

In [ ]:
df_train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
df_train.head(3)

Check shape of train data set i.e columns and rows count.

In [ ]:
df_train.shape

Check shape of test data set i.e columns and rows count.

In [ ]:
df_test.shape

In [ ]:
df_train.info()

Check if any columns have missing values.

In [ ]:
df_train.columns[df_train.isnull().any()]

Check the percentage of missing values in these columns.

In [ ]:
#missing value counts in each of these columns
miss = df_train.isnull().sum()/len(df_train)
miss = miss[miss > 0]
miss.sort_values(inplace=True)
miss

Visualising and plot the missing value count.

In [ ]:
df_miss = miss.to_frame()

In [ ]:
df_miss.columns = ['Mean(count)']
df_miss.index.names = ['Name']
df_miss['Name'] = df_miss.index

In [ ]:
sns.set(style="whitegrid", color_codes=True)
sns.barplot(x = 'Name', y = 'Mean(count)', data=df_miss)
plt.xticks(rotation = 90)
plt.show()

Proceed and check the distribution of the target variable.

In [ ]:
#SalePrice
sns.distplot(df_train['SalePrice'])

In [ ]:
#skewness
print("The skewness of SalePrice is {}".format(df_train['SalePrice'].skew()))

The target variable SalePrice has a right-skewed distribution. We'll need to log transform this variable so that it becomes normally distributed.

In [ ]:
#now transforming the target variable
target = np.log(df_train['SalePrice'])
print ('Skewness is', target.skew())
sns.distplot(target)

In [ ]:
#separate variables into new data frames
numeric_data = df_train.select_dtypes(include=[np.number])
cat_data = df_train.select_dtypes(exclude=[np.number])

In [ ]:
## Count of numerical columns 
numeric_data.shape

In [ ]:
cat_data.shape

In [ ]:
del numeric_data['Id']

In [ ]:
#correlation plot
corr = numeric_data.corr()
plt.subplots(figsize=(20,15))
sns.heatmap(corr, cmap='coolwarm')

In [ ]:
print (corr['SalePrice'].sort_values(ascending=False)[:15], '\n') #top 15 values
print ('----------------------')
print (corr['SalePrice'].sort_values(ascending=False)[-5:]) #last 5 values